Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>


<hr>

<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [46]:
import pandas as pd
from math import sqrt


Now let's read each file into their Dataframes:


In [47]:
# Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('../../data/raw/movies.csv')
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Let's also take a peek at how each of them are organized:


In [48]:
# Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('../../data/raw/ratings_sample.csv')
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01#pandas.Series.str.extract) function that Pandas has.


Let's remove the year from the **title** column by using pandas' replace function and store in a new **year** column.


In [49]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()


/tmp/ipykernel_11613/6725391.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.


In [50]:
# Dropping the genres column
movies_df = movies_df.drop('genres', 1)


/tmp/ipykernel_11613/4203219065.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movies_df = movies_df.drop('genres', 1)


Here's the final movies dataframe:


In [51]:
movies_df.head()


,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


<br>


Next, let's look at the ratings dataframe.


In [52]:
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.


In [53]:
# Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()


/tmp/ipykernel_11613/1176426921.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>

# Collaborative Filtering


Now it's time to start our work on recommendation systems.

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

*   Select a user with the movies the user has watched
*   Based on his rating to movies, find the top X neighbours
*   Get the watched movie record of the user for each neighbour.
*   Calculate a similarity score using some formula
*   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [54]:
user_input = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
input_movies = pd.DataFrame(user_input)
input_movies


,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [55]:
# Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(input_movies['title'].tolist())]
# Then merging it so we can get the movieId. It's implicitly merging it by title.
input_movies = pd.merge(inputId, input_movies)
# Dropping information we won't use from the input dataframe
input_movies = input_movies.drop('year', 1)
# Final input dataframe
# If a movie you added in above isn't here, then it might not be in the original 
# dataframe or it might spelled differently, please check capitalisation.
input_movies


/tmp/ipykernel_11613/813628688.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  input_movies = input_movies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#### The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [56]:
# Filtering out users that have watched movies that the input has watched and storing it
user_subset = ratings_df[ratings_df['movieId'].isin(input_movies['movieId'].tolist())]
user_subset.head()


,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


We now group up the rows by user ID.


In [57]:
# Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
user_subset_group = user_subset.groupby(['userId'])
user_subset_group.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
3899049,42118,296,3.0
3899165,42121,1968,4.0
3899433,42127,296,4.5
3899764,42128,2,3.0


Let's look at one of the users, e.g. the one with userID=1130.


In [58]:
user_subset_group.get_group(1130)


,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [59]:
# Sorting it so users with movie most in common with the input will have priority
user_subset_group = sorted(user_subset_group,  key=lambda x: len(x[1]), reverse=True)


/tmp/ipykernel_11613/2174774839.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  user_subset_group = sorted(user_subset_group,  key=lambda x: len(x[1]), reverse=True)


Now let's look at the first user.


In [60]:
user_subset_group[0:3]


[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.\
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.


In [61]:
user_subset_group = user_subset_group[0:100]


Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.


In [68]:
# Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearson_correlation_dict = {}

# For every user group in our subset
for name, group in user_subset_group:
    # Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    input_movies = input_movies.sort_values(by='movieId')
    # Get the N for the formula
    n_ratings = len(group)
    # Get the review scores for the movies that they both have in common
    temp_df = input_movies[input_movies['movieId'].isin(group['movieId'].tolist())]
    # And then store them in a temporary buffer variable in a list format to facilitate future calculations
    temp_rating_list = temp_df['rating'].tolist()
    # Let's also put the current user group reviews in a list format
    temp_group_list = group['rating'].tolist()
    # Now let's calculate the pearson correlation between two users, so called, x and y
    sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list),2)/float(n_ratings)
    syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list),2)/float(n_ratings)
    sxy = sum( i*j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list)*sum(temp_group_list)/float(n_ratings)
    
    # If the denominator is different than zero, then divide, else, 0 correlation.
    if (sxx != 0 and syy != 0) and ((sxx * sxy) > 0):
        pearson_correlation_dict[name] = sxy / sqrt(sxx * sxy)
    else:
        pearson_correlation_dict[name] = 0


In [69]:
pearson_correlation_dict.items()


dict_items([(75, 0.6504436355879909), (106, 0.43852900965351466), (686, 0.6793662204867574), (815, 0.6504436355879909), (1040, 0.8320502943378436), (1130, 0.6201736729460423), (1502, 0.6793662204867574), (1599, 0.3922322702763681), (1625, 0.5547001962252291), (1950, 0.2773500981126146), (2065, 0.3922322702763681), (2128, 0.6201736729460423), (2432, 0.2773500981126146), (2791, 0.6793662204867574), (2839, 0.7337993857053429), (2948, 0), (3025, 0.48038446141526137), (3040, 0.6201736729460423), (3186, 0.75955452531275), (3271, 0.43852900965351466), (3429, 0), (3734, 0), (4099, 0.19611613513818404), (4208, 0.48038446141526137), (4282, 0), (4292, 0.6504436355879909), (4415, 0), (4586, 0), (4725, 0), (4818, 0.6793662204867574), (5104, 1.0377490433255416), (5165, 0), (5547, 0.2773500981126146), (6082, 0), (6207, 0.9805806756909201), (6366, 0.6793662204867574), (6482, 0), (6530, 0), (7235, 0.898717034272917), (7403, 0.19611613513818404), (7641, 0.7844645405527362), (7996, 0.7337993857053429), (

In [70]:
pearson_df = pd.DataFrame.from_dict(pearson_correlation_dict, orient='index')
pearson_df.columns = ['similarityIndex']
pearson_df['userId'] = pearson_df.index
pearson_df.index = range(len(pearson_df))
pearson_df.head()


,similarityIndex,userId
0,0.650444,75
1,0.438529,106
2,0.679366,686
3,0.650444,815
4,0.832050,1040


#### The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.


In [71]:
top_users=pearson_df.sort_values(by='similarityIndex', ascending=False)[0:50]
top_users.head()


,similarityIndex,userId
62,1.091928,12120
75,1.037749,14529
30,1.037749,5104
34,0.980581,6207
55,0.980581,10707


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.


In [72]:
top_users_rating=top_users.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
top_users_rating.head()


,similarityIndex,userId,movieId,rating
0,1.091928,12120,1,1.5
1,1.091928,12120,2,1.0
2,1.091928,12120,4,3.0
3,1.091928,12120,9,1.0
4,1.091928,12120,10,3.0


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:


In [73]:
# Multiplies the similarity by the user's ratings
top_users_rating['weightedRating'] = top_users_rating['similarityIndex'] * top_users_rating['rating']
top_users_rating.head()


,similarityIndex,userId,movieId,rating,weightedRating
0,1.091928,12120,1,1.5,1.637893
1,1.091928,12120,2,1.0,1.091928
2,1.091928,12120,4,3.0,3.275785
3,1.091928,12120,9,1.0,1.091928
4,1.091928,12120,10,3.0,3.275785


In [74]:
# Applies a sum to the topUsers after grouping it up by userId
temp_top_users_rating = top_users_rating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
temp_top_users_rating.columns = ['sum_similarityIndex','sum_weightedRating']
temp_top_users_rating.head()


,sum_similarityIndex,sum_weightedRating
movieId,,
1,37.097693,134.015792
2,37.097693,87.236534
3,11.011501,27.836903
4,1.091928,3.275785
5,11.777032,27.014396


In [75]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = temp_top_users_rating['sum_weightedRating'] / temp_top_users_rating['sum_similarityIndex']
recommendation_df['movieId'] = temp_top_users_rating.index
recommendation_df.head()


,weighted average recommendation score,movieId
movieId,,
1,3.612510,1
2,2.351535,2
3,2.527984,3
4,3.000000,4
5,2.293820,5


Now let's sort it and see the top 20 movies that the algorithm recommended!


In [76]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)


,weighted average recommendation score,movieId
movieId,,
26444,5.0,26444
103210,5.0,103210
4334,5.0,4334
4347,5.0,4347
4356,5.0,4356
4422,5.0,4422
4426,5.0,4426
2171,5.0,2171
73824,5.0,73824


In [77]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]


,movieId,title,year
2088,2171,Next Stop Wonderland,1998
4240,4334,Yi Yi,2000
4253,4347,Divided We Fall (Musíme si pomáhat),2000
4262,4356,Gentlemen Prefer Blondes,1953
4328,4422,Cries and Whispers (Viskningar och rop),1972
4332,4426,Kiss Me Deadly,1955
8850,26444,"Hitch Hikers Guide to the Galaxy, The",1981
9064,26801,Dragon Inn (Sun lung moon hak chan),1992
14785,73824,Abigail's Party,1977
21258,103210,Fullmetal Alchemist: The Sacred Star of Milos,2011


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

*   Takes other user's ratings into consideration
*   Doesn't need to study or extract information from the recommended item
*   Adapts to the user's interests which might change over time

##### Disadvantages

*   Approximation function can be slow
*   There might be a low of amount of users to approximate
*   Privacy issues when trying to learn the user's preferences
